In [38]:
#%
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import tqdm
import cx_Oracle

connection = cx_Oracle.connect(user='scott', password='tiger', dsn='192.168.119.119:1521/dink11.dbsvr')
cursor = connection.cursor()
sql="select categorySmall, convenienceToEmbedding from CATEGORYembedding"
cursor.execute(sql)
rows = cursor.fetchall()
columns = ['categorySmall','convenienceToEmbedding']
category = pd.DataFrame(rows, columns=columns)

sql="SELECT c.email as email,ch.categorySmall, ch.amount FROM cardhistory ch JOIN card c ON ch.cardid = c.cardid WHERE ch.cardhisdate >= TO_DATE('2023-07-07', 'YYYY-MM-DD')"
cursor.execute(sql)
rows = cursor.fetchall()
columns = ['email','categorySmall','amount']
data = pd.DataFrame(rows, columns=columns)
data['categoryEmbedding'] = data['categorySmall'].replace(category.set_index('categorySmall')['convenienceToEmbedding'])
userList=data['email'].unique()

In [39]:
mode_card_df = {'email': [], 'categorySmall' : [], 'embedding_mode' : [],'amount':[]}
for i, userId in tqdm.tqdm(enumerate(userList)):
    '''
    _df : 카드 거래 내역 중 카드번호 같은 것 끼리 모은 임시 df
    mean_card_df : 카드번호 같은 것들 중, 최빈값 업종을 뽑아서 대표 업종으로 만든 df
    '''
    _df = data[data['email']==userId][['categorySmall', 'categoryEmbedding','amount']]
    # print(_df)
    # print(_df['categorySmall'].mode().iloc[0])
    # print(_df['amount'].mean())

    mode_card_df['email'].append(userId)
    mode_card_df['categorySmall'].append(_df['categorySmall'].mode().iloc[0])
    mode_card_df['embedding_mode'].append(_df['categoryEmbedding'].mode().iloc[0])
    mode_card_df['amount'].append(_df['amount'].mean())
    
# mode_card_df = pd.DataFrame(mode_card_df)
# print(mode_card_df)
print(len(mode_card_df['email']))
print(len(mode_card_df['categorySmall']))
print(len(mode_card_df['amount']))
mode_card_df = pd.DataFrame(mode_card_df)
mode_card_df['amount'] = mode_card_df['amount'].astype(int)
mode_card_df

0it [00:00, ?it/s]

2503it [01:27, 28.49it/s]

2503
2503
2503


,email,categorySmall,embedding_mode,amount
0,pooh5045@naver.com,한식,0.178922,47435
1,boramgim@gmail.com,일반병원,0.181306,84146
2,ihaeun@daum.net,일반병원,0.181306,78753
3,tson@naver.com,종합병원,0.229899,59513
4,juweongim@live.com,취미/오락,0.123046,112264
...,...,...,...,...
2498,jihyegang@dreamwiz.com,제과점,0.432291,76515
2499,jeongjai@naver.com,약국,0.391269,76120
2500,sunja84@hanmail.net,일식/중식/양식,0.212529,119173
2501,yunjihu@hanmail.net,제과점,0.372090,102954


In [40]:
from sklearn.cluster import KMeans

n_clusters = 4
X = np.array(list(zip(mode_card_df['embedding_mode'].values, mode_card_df['amount'].values)))
print(X.shape)

kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(X)
cluster_labels = kmeans.labels_
print(len(cluster_labels))
mode_card_df['Cluster'] = cluster_labels
mode_card_df['Cluster']=mode_card_df['Cluster'].astype(str)

(2503, 2)
2503


In [41]:
mode_card_df['Cluster'].value_counts()

3    894
0    867
1    534
2    208
Name: Cluster, dtype: int64

In [42]:
connection = cx_Oracle.connect(user='scott', password='tiger', dsn='192.168.119.119:1521/dink11.dbsvr')
cursor = connection.cursor()
insertSql="INSERT INTO clusterTable (email,clusterNum) values(:1,:2)"
for i in range(len(mode_card_df)):
    cursor.execute(insertSql, (mode_card_df.loc[i,'email'],mode_card_df.loc[i,'Cluster']))
connection.commit()
cursor.close()
connection.close()  

IntegrityError: ORA-00001: 무결성 제약 조건(SCOTT.SYS_C008520)에 위배됩니다

In [44]:
cursor.close()
connection.close()  

InterfaceError: not open